In [26]:
import json

# Load your training data
with open("train_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)


for record in data:
    text = record["text"]
    # for idx in range(len(text)):
    #     print(f"{idx:3}: {repr(text[idx])}")
    for entity in record["entities"]:
        start, end, label = entity
        print(f"{start:3}-{end:3}: {repr(text[start:end])} ({label})")

135-166: 'GPU Compiler Engineer - 3077321' (ROLE)
191-199: 'Qualcomm' (ORG)
100-134: 'Machine Learning Engineer (Remote)' (ROLE)
 59- 67: 'Weedmaps' (ORG)
138-146: 'Weedmaps' (ORG)
  0-  9: 'Microsoft' (ORG)
149-158: 'Microsoft' (ORG)
204-213: 'Microsoft' (ORG)
 98-136: 'Firmware Engineer (Job number:1847616)' (ROLE)
 50- 58: 'DoorDash' (ORG)
 61- 69: 'DoorDash' (ORG)
111-119: 'DoorDash' (ORG)
122-195: 'Software Engineer I, Entry-Level (Graduation Date: Fall 2024-Summer 2025)' (ROLE)
  0-  7: 'Spotify' (ORG)
 41- 48: 'Spotify' (ORG)
 99-127: 'Machine Learning Engineer II' (ROLE)
 52- 65: 'Creospan Inc.' (ORG)
 85- 98: 'Creospan Inc.' (ORG)
 31- 48: 'Software Engineer' (ROLE)
 44- 49: 'Adobe' (ORG)
135-184: 'R150523 2025 University Graduate - AI/ML Engineer' (ROLE)
 11- 36: 'Brains & Motion Education' (ORG)
153-187: 'INSTRUCTOR - AFTER-SCHOOL PROGRAMS' (ROLE)
 42- 47: 'Shell' (ORG)
178-183: 'Shell' (ORG)
365-370: 'Shell' (ORG)
 62-113: 'R182116 Shell Graduate Program 2026 - United State

In [ ]:
import pickle
from bs4 import BeautifulSoup
import re


with open("messages_cache.pkl", "rb") as f:
    loaded_messages = pickle.load(f)

def strip_html(html):
    soup = BeautifulSoup(html, "html.parser")
    
    for element in soup(['script', 'style', 'meta', 'link', 'title', 'head']):
        element.decompose()
    
    text = soup.get_text(separator="\n", strip=True)
    
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [1]:
import json
import spacy
from spacy.training.example import Example

nlp = spacy.load("en_core_web_md")

In [2]:
nlp.to_disk('models/en_core_web_md')

In [ ]:
import datetime
from datetime import timezone

all_messages = dict(
    sorted(
        loaded_messages.items(),
        key=lambda item: item[1]["date"] or datetime.min.replace(tzinfo=timezone.utc)
    )
)
print(len(all_messages))

276


In [41]:
import itertools
rolecount = 0
orgcount = 0
completecount = 0
onlyOrg = []
onlyRole = []
nothing = []

first_twenty = dict(itertools.islice(all_messages.items(), 20))
for key, msg in first_twenty.items():
    org = ""
    role = ""
    
    sender_preview = strip_html(msg['sender'])
    subject_preview = strip_html(msg['subject'])
    body_preview = strip_html(msg['body'])[:300]

    text = sender_preview + "|" + subject_preview + "|" + body_preview

    # ROLE extraction from subject+body
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "ROLE":
            role = ent.text
        if ent.label_ == "ORG" or ent.label_ == "GPE":
            org = ent.text
    if role:
        rolecount += 1
    if org:
        orgcount += 1
    if role and org:
        completecount += 1

    # print(f"emailID: {key}, ORG: {org}, ROLE: {role}")
    if org and not role:
        onlyOrg.append(msg)
    elif role and not org:
        onlyRole.append(msg)
    elif not role and not org:
        nothing.append(msg)
    
    print(f"emailID: {key}, ORG: {org}, ROLE: {role}")
    print(f"text: {text}")
    print("-" * 40)

print(f"Total messages: {len(loaded_messages)}")
print(f"Messages with ORG: {orgcount}")
print(f"Messages with ROLE: {rolecount}")
print(f"Messages with both ORG and ROLE: {completecount}")
print(f"Messages with none: {len(nothing)}")

emailID: 1982f7fb16f36fef, ORG: Qualcomm, ROLE: CAD Software Applications Engineer - 3077717
text: qualcomm@myworkday.com|Successfully submitted application for Ricky Li|Hi Ricky Li, We have received your application for position CAD Software Applications Engineer - 3077717 . Thank you for choosing Qualcomm as a prospective employer, where we are passionate about our people and the limitless potential of your career. We are currently reviewing your application and w
----------------------------------------
emailID: 19834486301c52fd, ORG: Qualcomm, ROLE: GPU Software Engineer - 3078425
text: qualcomm@myworkday.com|Successfully submitted application for Ricky Li|Hi Ricky Li, We have received your application for position GPU Software Engineer - 3078425 . Thank you for choosing Qualcomm as a prospective employer, where we are passionate about our people and the limitless potential of your career. We are currently reviewing your application and will reach out
------------------------------

In [9]:
for msg in onlyRole:
    sender_preview = strip_html(msg['sender'])
    subject_preview = strip_html(msg['subject'])
    body_preview = strip_html(msg['body'][:300])
    
    sender_subject = (sender_preview + " | " + subject_preview)
    subject_body = (subject_preview + " | " + body_preview)

    sender_subject = re.sub(r"\s+", " ", sender_subject).strip()
    subject_body = re.sub(r"\s+", " ", subject_body).strip()

    print(f"emailID: {key}")
    print(f"Sender+Subject: {sender_subject}")
    print(f"Subject+Body: {subject_body}")
    print("=" * 40)

emailID: 19839503b19a2075
Sender+Subject: NokiaME DoNotReply | Your recent job application for Hardware Engineer - 19497
Subject+Body: Your recent job application for Hardware Engineer - 19497 | Dear Ricky, We received your job application for Hardware Engineer - 19497. We will review your application and get back to you soon. If you were requested to provide additional info about your job application, or if you want to manage your profile, go
